In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import sklearn as sk
from sklearn import ensemble
import os

In [2]:
pd.options.mode.use_inf_as_na = True

In [3]:
DATA_FOLDER = '/home/roger/code/kaggle/predict-future-sales/data/'
PRED_FOLDER = '/home/roger/code/kaggle/predict-future-sales/predictions/'

transactions    = pd.read_csv(DATA_FOLDER + 'sales_train.csv')
items           = pd.read_csv(DATA_FOLDER + 'items.csv')
item_categories = pd.read_csv(DATA_FOLDER + 'item_categories_en.csv')
shops           = pd.read_csv(DATA_FOLDER + 'shops_en.csv')
test            = pd.read_csv(DATA_FOLDER + 'test.csv')
sample          = pd.read_csv(DATA_FOLDER + 'sample_submission.csv')

In [4]:
train = (transactions
     .merge(items, on="item_id")
     .merge(item_categories, on="item_category_id")
     .merge(shops, on="shop_id"))
train["date"] = pd.to_datetime(train["date"], format="%d.%m.%Y")

In [5]:
train.head().T

,0,1,2,3,4
date,2013-01-02 00:00:00,2013-04-26 00:00:00,2013-06-26 00:00:00,2013-07-20 00:00:00,2013-09-14 00:00:00
date_block_num,0,3,5,6,8
shop_id,59,59,59,59,59
item_id,22154,944,944,944,944
item_price,999.0,150.0,199.5,199.5,299.0
item_cnt_day,1.0,1.0,1.0,1.0,2.0
item_name,ЯВЛЕНИЕ 2012 (BD),2012 (BD),2012 (BD),2012 (BD),2012 (BD)
item_category_id,37,37,37,37,37
item_category_name,Cinema - Blu-Ray,Cinema - Blu-Ray,Cinema - Blu-Ray,Cinema - Blu-Ray,Cinema - Blu-Ray
shop_name,"Yaroslavl TC ""Altair""","Yaroslavl TC ""Altair""","Yaroslavl TC ""Altair""","Yaroslavl TC ""Altair""","Yaroslavl TC ""Altair"""


## Predicts sales with most recent months sales

In [16]:
X = (transactions[["date_block_num", "shop_id", "item_id", "item_cnt_day"]]
     .groupby(["date_block_num", "shop_id", "item_id"])
     .agg("sum")
     .reset_index()
    )
X.head()

,date_block_num,shop_id,item_id,item_cnt_day
0,0,0,32,6.0
1,0,0,33,3.0
2,0,0,35,1.0
3,0,0,43,1.0
4,0,0,51,2.0


In [17]:
X["date_block_num"].max()

33

In [21]:
sample1 = sample.merge(test, on="ID")
sample2 = sample1.merge(
    X[X["date_block_num"]==33].drop(columns="date_block_num"),
    how="left",
    on=["shop_id", "item_id"]
)

In [24]:
sample2["item_cnt_month"] = (
    sample2["item_cnt_day"]
    .apply(lambda x:0 if np.isnan(x) else x)
)

In [26]:
sample2[["ID", "item_cnt_month"]].to_csv("comp_submissions/month33.csv", index=False)

In [27]:
sample2.shape

(214200, 5)